In [39]:
import pandas as pd, numpy as np
import matplotlib, seaborn as sns
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (7,3)
plt.rcParams['figure.constrained_layout.use'] = True

In [82]:
# load data and crop to earliest_all and latest_all
data_list = {}
earliest_all = '2015-08-07'
latest_all = '2020-06-26'
mydateparser = lambda x: pd.datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S')
mydateparser1 = lambda x: pd.datetime.strptime(str(x), '%Y-%m-%d')
btc_series = pd.read_excel('Data/BTC_closing.xlsx',squeeze=True, parse_dates=[0], index_col=0, date_parser=mydateparser)[earliest_all:latest_all]
data_list['btc_series'] = btc_series
eth_series = pd.read_excel('Data/ETH.xlsx',squeeze=True, parse_dates=[0], index_col=0, date_parser=mydateparser, usecols='A,E')[earliest_all:latest_all]
data_list['eth_series'] = eth_series

oil_WTI = pd.read_excel('Data/DCOILWTICO.xls', squeeze=True, parse_dates=[0], index_col=0, skiprows=10, date_parser=mydateparser)[earliest_all:latest_all]
data_list['oil_WTI'] = oil_WTI
oil_BRENT = pd.read_excel('Data/DCOILBRENTEU.xls', squeeze=True, parse_dates=[0], index_col=0, skiprows=10, date_parser=mydateparser)[earliest_all:latest_all]
data_list['oil_BRENT'] = oil_BRENT

ex_JPN_USD = pd.read_excel('Data/DEXJPUS.xls', squeeze=True, parse_dates=[0], index_col=0, skiprows=10, date_parser=mydateparser)[earliest_all:latest_all]
data_list['ex_JPN_USD'] = ex_JPN_USD
ex_EUR_USD = 1 / pd.read_excel('Data/DEXUSEU.xls', squeeze=True, parse_dates=[0], index_col=0, skiprows=10, date_parser=mydateparser)[earliest_all:latest_all]
data_list['ex_EUR_USD'] = ex_EUR_USD
ex_GBP_USD = 1 / pd.read_excel('Data/DEXUSUK.xls', squeeze=True, parse_dates=[0], index_col=0, skiprows=10, date_parser=mydateparser)[earliest_all:latest_all]
data_list['ex_GBP_USD'] = ex_GBP_USD

gold_10am = pd.read_excel('Data/GOLDAMGBD228NLBM10AM.xls', squeeze=True, parse_dates=[0], index_col=0, skiprows=10, date_parser=mydateparser)[earliest_all:latest_all]
data_list['gold_10am'] = gold_10am
gold_03pm = pd.read_excel('Data/GOLDPMGBD228NLBM3PM.xls', squeeze=True, parse_dates=[0], index_col=0, skiprows=10, date_parser=mydateparser)[earliest_all:latest_all]
data_list['gold_03pm'] = gold_03pm

dax = pd.read_excel('Data/SDAXI.xlsx', squeeze=True, parse_dates=[0], index_col=0, date_parser=mydateparser1, usecols=[0,5])[earliest_all:latest_all]
data_list['dax'] = dax
sp500 = pd.read_excel('Data/SP500.xls', squeeze=True, parse_dates=[0], index_col=0, skiprows=10, date_parser=mydateparser)[earliest_all:latest_all]
data_list['sp500'] = sp500

In [85]:
# get series overview: same start and end but different lengths, because some days are missing
def print_data_list():
    for i in data_list:
        if i == 'dax' or i == 'sp500': 
            t = 4
        else: 
            t = 3
        print(i, '\t'*t, 'n =' , len(data_list[i]), '\nFirst:\t', 
              data_list[i].index[0], '\nLast:\t',
              data_list[i].index[-1], '\n')

print_data_list()

btc_series 			 n = 1786 
First:	 2015-08-07 00:00:00 
Last:	 2020-06-26 00:00:00 

eth_series 			 n = 1786 
First:	 2015-08-07 00:00:00 
Last:	 2020-06-26 00:00:00 

oil_WTI 			 n = 1786 
First:	 2015-08-07 00:00:00 
Last:	 2020-06-26 00:00:00 

oil_BRENT 			 n = 1786 
First:	 2015-08-07 00:00:00 
Last:	 2020-06-26 00:00:00 

ex_JPN_USD 			 n = 1786 
First:	 2015-08-07 00:00:00 
Last:	 2020-06-26 00:00:00 

ex_EUR_USD 			 n = 1786 
First:	 2015-08-07 00:00:00 
Last:	 2020-06-26 00:00:00 

ex_GBP_USD 			 n = 1786 
First:	 2015-08-07 00:00:00 
Last:	 2020-06-26 00:00:00 

gold_10am 			 n = 1786 
First:	 2015-08-07 00:00:00 
Last:	 2020-06-26 00:00:00 

gold_03pm 			 n = 1786 
First:	 2015-08-07 00:00:00 
Last:	 2020-06-26 00:00:00 

dax 				 n = 1786 
First:	 2015-08-07 00:00:00 
Last:	 2020-06-26 00:00:00 

sp500 				 n = 1786 
First:	 2015-08-07 00:00:00 
Last:	 2020-06-26 00:00:00 



In [84]:
# for each series, resample to get every day between start and end period
# for each series, fill newly added days with NaN, then interpolate (method=time)

def fill_in_missing_data(data_list):
    for i in data_list:
        # resample
        new_i = data_list[i].resample('D').sum()
        # replace 0 with NaN
        new_i.replace(0, np.nan, inplace=True)
        # interpolate (time)
        new_i.interpolate(method='time', inplace=True)
        data_list[i] = new_i

fill_in_missing_data(data_list)
print_data_list()

btc_series 	  is equal to dax
eth_series 	  is equal to dax
oil_WTI 	  is equal to dax
oil_BRENT 	  is equal to dax
ex_JPN_USD 	  is equal to dax
ex_EUR_USD 	  is equal to dax
ex_GBP_USD 	  is equal to dax
gold_10am 	  is equal to dax
gold_03pm 	  is equal to dax
dax 		  is equal to dax
sp500 		  is equal to dax
